# Begin
## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyedflib 
import pandas as pd
import mne

In [5]:
!pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:009.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


## Dowload data set 

In [2]:
# !wget -r -N -c -np https://physionet.org/files/eegmmidb/1.0.0/

## Positioning of electrodes on the head

In [3]:
img = plt.imread('files/64_channel_sharbrough-old.png')
plt.imshow(img)
plt.axis('off')
plt.show()


# Convert from file to data frame

In [4]:
def file_to_DataDrame(path):
    """
    This function takes in a file path and returns a dataframe with the data and the target values
    format:
        Fc5	        Fc3	        Fc1	        ...	Oz	        O2	        Iz	        target
    0	-0.000046	-0.000041	-0.000032	...	0.000040	0.000108	0.000055	T0
    1	-0.000054	-0.000048	-0.000034	...	0.000064	0.000114	0.000074	T0
    ...
    """

    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "")
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    return df

In [5]:
subject = 1
file = 5
fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"

df = file_to_DataDrame(fileName)

In [6]:
df.head(2)

In [7]:
df.tail(2)

In [8]:
def index_of_TX(df):
    key = 0
    dict_TX_index = {key: [df['target'][0], 0] }
    current_TX = df['target'][0]
    for i in range(len(df['target'])):
        if df['target'][i] != current_TX:
            key += 1
            dict_TX_index[key] = [df['target'][i], i]
            current_TX = df['target'][i]
 
    return dict_TX_index
   
        
print(index_of_TX(df))

In [9]:
indexs = index_of_TX(df)
xT0=[]
xT1=[]
xT2=[]
ax= df.plot(y='Fc5')
ax.set_xlabel("Number of samples")
for i in range(0,len(indexs)):
    if indexs[i][0] == 0:
        xT0.append(indexs[i][1])      
    elif indexs[i][0] == 1:
        xT1.append(indexs[i][1])
    elif indexs[i][0] == 2:
        xT2.append(indexs[i][1])
#doter line
ax.vlines(x=xT0, ymin=-0.0003, ymax=0.0003, color='r',label='T0',linestyles='dashed')
ax.vlines(x=xT1, ymin=-0.0003, ymax=0.0003, color='b',label='T1',linestyles='dashed')
ax.vlines(x=xT2, ymin=-0.0003, ymax=0.0003, color='y',label='T2', linestyles='dashed')
ax.legend()



# Colect data in one data set

In [10]:
def all_exp_to_array(num_person, choose_num_of_exp):
    """
    num_person: int renage(1,109)
    choose_num_of_exp: list of int[3,4,7,8,11,12]

    return: all_meseurments, all_targets
    """
   

    all_meseurments = np.zeros((len(choose_num_of_exp),20000,64))
    all_targets = np.zeros((len(choose_num_of_exp),20000))
    for i in range(len(choose_num_of_exp)):
        fileName = f"files/S{num_person:03d}/S{num_person:03d}R{choose_num_of_exp[i]:02d}.edf"
        df = file_to_DataDrame(fileName)
        for j in range(20000):
            if j >= len(df):
                all_meseurments[i][j] = np.zeros(64) 
                all_targets[i][j] = np.nan
            else:
                all_meseurments[i][j] = df.iloc[j].values[:-1]
                all_targets[i][j] = df.iloc[j].values[-1]
    return all_meseurments, all_targets
                
      
          

In [11]:
def all_files_to_array(choose_num_of_exp):
    """
    num_person: int 
    choose_num_of_exp: list of int [3,4,7,8,11,12]

    return: all_meseurments, all_targets
    """
    all_meseurments = np.zeros((109,len(choose_num_of_exp),20000,64))
    all_targets = np.zeros((109,len(choose_num_of_exp),20000))
    for i in range(1, 109):
        all_meseurments[i], all_targets[i] = all_exp_to_array(i, choose_num_of_exp)
    return all_meseurments, all_targets

In [ ]:
all_meseurments, all_targets = all_files_to_array([3,4,7,8,11,12])